

---



---


# 📌 **Regression Models**


---



---


Tujuan notebook ini:
- Membangun pipeline regresi end-to-end untuk memprediksi target kontinu dari fitur numerik.
- Mencakup: load data, EDA singkat, preprocessing (imputasi, outlier handling, scaling),
  train/val split, baseline Linear Regression, RandomForest, XGBoost, hyperparameter tuning,
  stacking sederhana, evaluasi akhir, saving model & inference.

Dataset:
- File: /content/midterm-regresi-dataset.csv
- Format: baris pertama = target (kontinu), diikuti ~89 fitur numerik tanpa header.



# 1. Install & import library
>Kode ini memuat berbagai library penting untuk analisis data, visualisasi, dan pemodelan regresi. Pandas dan NumPy digunakan untuk manipulasi data, sedangkan matplotlib dan seaborn untuk visualisasi. Modul scikit-learn disertakan untuk preprocessing, pembagian data, dan penerapan berbagai model regresi seperti Linear, Ridge, Lasso, Random Forest, Gradient Boosting, dan Decision Tree. XGBoost juga ditambahkan sebagai model boosting lanjutan. Berbagai metrik evaluasi diimpor untuk menilai performa model, dan random seed diatur agar hasil dapat direproduksi.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Evaluation metrics
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error
)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")

# 2. Load Dataset
>Kode ini digunakan untuk memuat dataset pelatihan dari file train_transaction.csv dan menampilkan informasi penting tentang isinya. Setelah data dibaca dengan pd.read_csv, program mencetak ukuran dataset, contoh lima baris pertama, statistik deskriptif untuk variabel target TransactionAmt, serta jumlah total nilai yang hilang di seluruh kolom. Langkah ini membantu memahami struktur awal data dan kondisi kualitas dataset sebelum dilakukan proses analisis atau preprocessing lebih lanjut.

In [ ]:
# Load the training data
train_df = pd.read_csv('datasets/transaction/train_transaction.csv')

print("Dataset Shape:", train_df.shape)
print("\n" + "="*80)
print("First 5 rows:")
print(train_df.head())
print("\n" + "="*80)
print("Target Variable Statistics (TransactionAmt):")
print(train_df['TransactionAmt'].describe())
print("\n" + "="*80)
print("Missing Values:")
print(f"Total missing values: {train_df.isnull().sum().sum()}")

# 3. Exploratory Data Analysis (EDA)
>Kode ini digunakan untuk memvisualisasikan distribusi nilai TransactionAmt menggunakan tiga jenis grafik: histogram, boxplot, dan histogram versi log-transform. Histogram pertama menunjukkan persebaran nilai transaksi dalam rentang umum, sementara boxplot membantu melihat penyebaran dan potensi outlier. Grafik log-transform digunakan untuk membuat distribusi yang sangat miring menjadi lebih mudah dianalisis. Setelah visualisasi ditampilkan, kode juga mencetak rentang nilai transaksi, rata-rata, dan median untuk memberikan gambaran statistik dasar tentang variabel tersebut.

In [ ]:
# Visualize Transaction Amount Distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(train_df['TransactionAmt'], bins=100, color='skyblue', edgecolor='black')
axes[0].set_title('Transaction Amount Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Transaction Amount ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_xlim(0, 1000)  # Focus on common range

# Box plot
axes[1].boxplot(train_df['TransactionAmt'], vert=True)
axes[1].set_title('Transaction Amount Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Transaction Amount ($)')
axes[1].set_ylim(0, 500)

# Log scale distribution
axes[2].hist(np.log1p(train_df['TransactionAmt']), bins=100, color='lightcoral', edgecolor='black')
axes[2].set_title('Log-Transformed Transaction Amount', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Log(Transaction Amount)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\nTransaction Amount Range: ${train_df['TransactionAmt'].min():.2f} - ${train_df['TransactionAmt'].max():.2f}")
print(f"Mean Transaction Amount: ${train_df['TransactionAmt'].mean():.2f}")
print(f"Median Transaction Amount: ${train_df['TransactionAmt'].median():.2f}")

# 4. Data Preprocessing and Feature Engineering
>Kode ini membuat salinan dataset dan menghapus kolom TransactionID serta isFraud karena tugas ini fokus pada regresi, bukan klasifikasi. Setelah itu, hanya fitur-fitur penting yang dipilih, lalu kolom kategorikal diubah menjadi format numerik menggunakan LabelEncoder. Untuk menangani nilai hilang, digunakan SimpleImputer dengan strategi median agar lebih tahan terhadap nilai ekstrem. Selanjutnya, dilakukan pembersihan data dengan menghapus outlier pada kolom TransactionAmt menggunakan metode IQR sehingga model menjadi lebih stabil. Hasil akhirnya adalah dataset yang telah diseleksi, di-encode, diimputasi, dan dibersihkan dari outlier, siap digunakan untuk pelatihan model.

In [ ]:
# Create a working copy
df = train_df.copy()

# Drop TransactionID and isFraud (for regression, we don't use fraud label)
df = df.drop(['TransactionID', 'isFraud'], axis=1, errors='ignore')

# Select important features for regression
important_cols = ['TransactionAmt', 'TransactionDT', 'ProductCD',
                  'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
                  'addr1', 'addr2', 'dist1', 'dist2',
                  'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
                  'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D15',
                  'M1', 'M2', 'M3', 'M4',
                  'V12', 'V13', 'V20', 'V36', 'V37', 'V45', 'V53', 'V54']

# Keep only columns that exist
important_cols = [col for col in important_cols if col in df.columns]
df = df[important_cols]

print(f"Selected {len(important_cols)} features for modeling")
print(f"Dataset shape after feature selection: {df.shape}")

# Handle categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'TransactionAmt' in categorical_cols:
    categorical_cols.remove('TransactionAmt')

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

print(f"\n✓ Categorical variables encoded")

# Handle missing values
print(f"\nMissing values before imputation: {df.isnull().sum().sum()}")
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print(f"Missing values after imputation: {df_imputed.isnull().sum().sum()}")

# Remove outliers using IQR method (optional - keeps extreme values reasonable)
Q1 = df_imputed['TransactionAmt'].quantile(0.25)
Q3 = df_imputed['TransactionAmt'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

df_clean = df_imputed[(df_imputed['TransactionAmt'] >= lower_bound) &
                      (df_imputed['TransactionAmt'] <= upper_bound)]

print(f"\nData shape after outlier removal: {df_clean.shape}")
print(f"Removed {len(df_imputed) - len(df_clean)} outliers")

# Separate features and target
>Kode ini memisahkan dataset menjadi fitur (X) dan target (y) dengan menjadikan TransactionAmt sebagai variabel yang ingin diprediksi. Setelah itu, data dibagi menjadi training set dan test set dengan proporsi 80:20 untuk memastikan model dapat diuji secara objektif. Langkah selanjutnya adalah melakukan feature scaling menggunakan StandardScaler agar seluruh fitur berada pada skala yang sama, sehingga model—terutama yang sensitif terhadap perbedaan skala—dapat belajar dengan lebih stabil dan efisien. Hasil akhirnya adalah data yang sudah terpisah, terstandardisasi, dan siap digunakan untuk pelatihan model regresi.

In [ ]:
# Separate features and target
X = df_clean.drop('TransactionAmt', axis=1)
y = df_clean['TransactionAmt']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Data split and scaled successfully")

# 5. Model Training and Evaluation
>Kode ini membuat sebuah fungsi untuk melatih dan mengevaluasi model regresi secara menyeluruh. Di dalamnya, model akan dilatih menggunakan data training lalu menghasilkan prediksi untuk data training dan data testing. Fungsi ini menghitung metrik penting seperti R², RMSE, dan MAE untuk menilai performa model pada kedua subset data. Selain evaluasi numerik, fungsi juga menampilkan dua visualisasi: grafik Predicted vs Actual untuk melihat sejauh mana hasil prediksi mendekati nilai sebenarnya, serta Residual Plot untuk memeriksa pola kesalahan model. Pada akhirnya, fungsi mengembalikan ringkasan performa dalam bentuk dictionary sehingga mudah dianalisis atau dibandingkan antar model.


In [ ]:
# Function to evaluate regression models
def evaluate_regression_model(name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate a regression model
    """
    print(f"\n{'='*80}")
    print(f"Training {name}...")
    print('='*80)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)

    print(f"\n📊 Training Performance:")
    print(f"   R² Score: {train_r2:.4f}")
    print(f"   RMSE:     ${train_rmse:.2f}")
    print(f"   MAE:      ${train_mae:.2f}")

    print(f"\n📊 Testing Performance:")
    print(f"   R² Score: {test_r2:.4f}")
    print(f"   RMSE:     ${test_rmse:.2f}")
    print(f"   MAE:      ${test_mae:.2f}")

    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Predicted vs Actual
    axes[0].scatter(y_test, y_pred_test, alpha=0.5, s=10)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[0].set_xlabel('Actual Transaction Amount ($)')
    axes[0].set_ylabel('Predicted Transaction Amount ($)')
    axes[0].set_title(f'{name} - Predicted vs Actual', fontweight='bold')
    axes[0].grid(alpha=0.3)

    # Residual plot
    residuals = y_test - y_pred_test
    axes[1].scatter(y_pred_test, residuals, alpha=0.5, s=10)
    axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[1].set_xlabel('Predicted Transaction Amount ($)')
    axes[1].set_ylabel('Residuals ($)')
    axes[1].set_title(f'{name} - Residual Plot', fontweight='bold')
    axes[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    return {
        'model': name,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae
    }

print("✓ Evaluation function defined")

# 5.1 Linear Regression
>Kode ini melatih model Linear Regression menggunakan data yang telah di-scaling. Model kemudian dievaluasi menggunakan fungsi evaluate_regression_model(), yang menghitung performa model pada data training dan testing serta menampilkan visualisasi prediksi dan residual. Tujuan langkah ini adalah menjadikan Linear Regression sebagai baseline sederhana sebelum dibandingkan dengan model regresi lainnya yang lebih kompleks.

In [ ]:
# Train Linear Regression
lr_model = LinearRegression()
lr_results = evaluate_regression_model('Linear Regression', lr_model,
                                       X_train_scaled, y_train,
                                       X_test_scaled, y_test)

# 5.2 Ridge Regression (L2 Regularization)
>Kode ini menjalankan pelatihan model Ridge Regression, yaitu varian dari regresi linear yang menggunakan regularisasi L2 untuk mencegah overfitting. Parameter alpha=1.0 menentukan seberapa kuat regularisasi diterapkan, sementara random_state=42 memastikan hasil yang konsisten. Model ini kemudian dievaluasi menggunakan fungsi evaluate_regression_model(), yang menghitung metrik performa seperti R², RMSE, dan MAE, serta menampilkan grafik prediksi dan residual. Ridge Regression biasanya memberikan hasil lebih stabil dibanding Linear Regression, terutama ketika fitur memiliki multikolinearitas atau varians tinggi.

In [ ]:
# Train Ridge Regression
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_results = evaluate_regression_model('Ridge Regression', ridge_model,
                                         X_train_scaled, y_train,
                                         X_test_scaled, y_test)

# 5.3 Lasso Regression (L1 Regularization)
>Kode ini melatih model Lasso Regression, yaitu metode regresi linear yang menggunakan regularisasi L1 untuk mengurangi overfitting sekaligus melakukan seleksi fitur secara otomatis. Dengan alpha=0.1, model mengontrol kekuatan penalti sehingga beberapa koefisien dapat dipaksa menjadi nol, membuat model lebih sederhana dan efisien. Parameter max_iter=10000 memastikan proses optimisasi berjalan hingga konvergen. Model kemudian dievaluasi menggunakan fungsi evaluate_regression_model(), yang menghitung metrik seperti R², RMSE, serta MAE, dan menghasilkan visualisasi prediksi serta residual. Lasso Regression biasanya efektif ketika sejumlah fitur tidak terlalu berpengaruh dan perlu dieliminasi.

In [ ]:
# Train Lasso Regression
lasso_model = Lasso(alpha=0.1, random_state=42, max_iter=10000)
lasso_results = evaluate_regression_model('Lasso Regression', lasso_model,
                                         X_train_scaled, y_train,
                                         X_test_scaled, y_test)

# 5.4 Random Forest Regressor
>Kode ini melatih model Random Forest Regressor, yaitu metode ensemble berbasis kumpulan decision tree yang mampu menangkap pola hubungan kompleks dalam data. Parameter seperti n_estimators=100 mengatur jumlah pohon, max_depth=15 membatasi kedalaman pohon untuk mencegah overfitting, dan n_jobs=-1 memungkinkan pemrosesan paralel agar pelatihan lebih cepat. Setelah pelatihan selesai, model dievaluasi menggunakan fungsi evaluate_regression_model(). Selain itu, kode juga menghitung feature importance untuk menunjukkan fitur mana yang paling berpengaruh dalam prediksi. Hasilnya divisualisasikan melalui grafik baris yang menampilkan 15 fitur terpenting, sehingga memudahkan analisis kontribusi tiap variabel.

In [ ]:
# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_results = evaluate_regression_model('Random Forest', rf_model,
                                      X_train_scaled, y_train,
                                      X_test_scaled, y_test)

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature', palette='viridis')
plt.title('Top 15 Most Important Features (Random Forest)', fontsize=14, fontweight='bold')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

# 5.5 Gradient Boosting Regressor
>Kode ini melatih model Gradient Boosting Regressor, yaitu algoritma boosting yang membangun pohon keputusan secara bertahap, di mana setiap pohon baru berfungsi memperbaiki kesalahan dari pohon sebelumnya. Parameter n_estimators=100 menentukan jumlah pohon, max_depth=5 mengatur kompleksitas tiap pohon, dan learning_rate=0.1 mengontrol seberapa besar kontribusi setiap pohon baru. Model kemudian dievaluasi menggunakan fungsi evaluate_regression_model(), yang menampilkan metrik performa dan grafik analisis. Gradient Boosting biasanya memberikan performa lebih baik dari metode linear karena mampu menangani hubungan non-linear secara efektif.

In [ ]:
# Train Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
gb_results = evaluate_regression_model('Gradient Boosting', gb_model,
                                      X_train_scaled, y_train,
                                      X_test_scaled, y_test)

# 5.6 XGBoost Regressor
>Kode ini melatih model XGBoost Regressor, yaitu salah satu algoritma gradient boosting tercepat dan paling efisien yang banyak digunakan untuk kompetisi machine learning. Dengan parameter n_estimators=100, model membangun 100 pohon secara bertahap, sementara max_depth=6 mengatur kedalaman setiap pohon agar model cukup fleksibel tanpa overfitting. Nilai learning_rate=0.1 mengontrol seberapa besar kontribusi tiap pohon baru dalam memperbaiki kesalahan prediksi sebelumnya. Setelah model selesai dilatih, performanya dievaluasi melalui fungsi evaluate_regression_model(), menghasilkan metrik seperti R², RMSE, MAE, serta visualisasi prediksi dan residual. Model XGBoost sangat efektif dalam menangani pola non-linear dan interaksi antar fitur dengan performa yang stabil dan cepat.

In [ ]:
# Train XGBoost Regressor
xgb_model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
xgb_results = evaluate_regression_model('XGBoost', xgb_model,
                                       X_train_scaled, y_train,
                                       X_test_scaled, y_test)

# 6. Model Comparison and Final Results
>Kode ini menyusun seluruh hasil evaluasi dari berbagai model regresi ke dalam satu DataFrame agar performanya mudah dibandingkan. Setelah itu, beberapa grafik dibuat untuk menampilkan perbandingan nilai R², RMSE, MAE, serta perbandingan R² antara data training dan testing sebagai indikasi potensi overfitting. Visualisasi ini membantu memahami model mana yang paling akurat dan paling stabil. Selanjutnya, kode secara otomatis memilih model terbaik berdasarkan nilai R² pada data testing, lalu menampilkan ringkasan performa model tersebut. Dengan cara ini, proses evaluasi menjadi lebih terstruktur, informatif, dan memudahkan pengambilan keputusan terkait model mana yang paling layak digunakan.

In [ ]:
# Compile all results
results_df = pd.DataFrame([lr_results, ridge_results, lasso_results,
                           rf_results, gb_results, xgb_results])
results_df = results_df.round(4)

print("="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# R² Score Comparison
axes[0, 0].bar(results_df['model'], results_df['test_r2'],
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[0, 0].set_title('Model R² Score Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate(results_df['test_r2']):
    axes[0, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# RMSE Comparison
axes[0, 1].bar(results_df['model'], results_df['test_rmse'],
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[0, 1].set_title('Model RMSE Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('RMSE ($)')
axes[0, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df['test_rmse']):
    axes[0, 1].text(i, v + 1, f'${v:.2f}', ha='center', fontweight='bold', fontsize=9)

# MAE Comparison
axes[1, 0].bar(results_df['model'], results_df['test_mae'],
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[1, 0].set_title('Model MAE Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('MAE ($)')
axes[1, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df['test_mae']):
    axes[1, 0].text(i, v + 0.5, f'${v:.2f}', ha='center', fontweight='bold', fontsize=9)

# Train vs Test R² (Overfitting check)
x = np.arange(len(results_df))
width = 0.35
axes[1, 1].bar(x - width/2, results_df['train_r2'], width, label='Train R²', alpha=0.8)
axes[1, 1].bar(x + width/2, results_df['test_r2'], width, label='Test R²', alpha=0.8)
axes[1, 1].set_title('Train vs Test R² Score', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(results_df['model'], rotation=45, ha='right')
axes[1, 1].legend()
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Identify best model
best_model_idx = results_df['test_r2'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'model']
best_r2 = results_df.loc[best_model_idx, 'test_r2']
best_rmse = results_df.loc[best_model_idx, 'test_rmse']
best_mae = results_df.loc[best_model_idx, 'test_mae']

print("\n" + "="*80)
print("🏆 BEST MODEL SELECTION")
print("="*80)
print(f"Best Model: {best_model_name}")
print(f"R² Score: {best_r2:.4f}")
print(f"RMSE: ${best_rmse:.2f}")
print(f"MAE: ${best_mae:.2f}")
print("="*80)
print("\n✅ Regression model training and evaluation completed successfully!")